In [1]:
import numpy as np
import glob
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pyemma
import msmbuilder
from msmbuilder.msm import MarkovStateModel
from msmbuilder.lumping import PCCAPlus

In [3]:
ls

cluster.py                      run-torque.sh
clustercenters.npy              samples_300_p1_20_100per.npy
dist_2500_unitime.ipynb         samples_300_p1_50_100per.npy
dtrajs.npy                      sanity_check_msm_300.png
its.png                         sanity_check_msm_300_p1.png
msm_300_p1_20_P.png             sanity_check_msm_300_p2.png
populations.txt                 unitime_2500_msm_300_p1_20.png
pyemma.log


In [2]:
dtrajs = list(np.load('dtrajs.npy', encoding='bytes'))

In [5]:
len(dtrajs)

3400

In [5]:
its = pyemma.msm.its(dtrajs)

23-02-17 18:29:05 pyemma.msm.estimators.implied_timescales.ImpliedTimescales[0] WARNING  Some timescales could not be computed. Timescales array is smaller than expected or contains NaNs


In [6]:
plt.figure()
pyemma.plots.plot_implied_timescales(its)
plt.savefig('its.png', dpi=300)
plt.close()

In [7]:
# lag 300

In [8]:
# MSMBUILDER ONLY - SAMPLES EQUAL AMOUNTS OF FRAMES / MACROSTATE; 
def macrostate_sample_equal(msmbuilder_msm, msmbuilder_pcca_object, n_samples_per_macrostate=100, dtrajs=None):
    import random
    import numpy as np
    
    # make dtrajs_dict
    dtrajs_dict = dict()
    traj_index = 0

    for traj in dtrajs:
        frame_index = 0
        for frame in traj:
            if frame in dtrajs_dict:
                dtrajs_dict[frame].append((traj_index, frame_index))
            else:
                dtrajs_dict[frame] = [(traj_index, frame_index)]
            frame_index += 1    
        traj_index += 1    

    metastable_sets = []
    samples = []
    
    # make mestable_sets
    for x in range(msmbuilder_pcca_object.n_macrostates):
        metastable_sets.append([])
    for x in range(len(msmbuilder_pcca_object.microstate_mapping_)):
        metastable_sets[msmbuilder_pcca_object.microstate_mapping_[x]].append(x)

    # sample
    for metastable_set in metastable_sets:
        all_frames = []
        for microstate in metastable_set:
            microstate_label = msmbuilder_msm.state_labels_[microstate]
            all_frames += dtrajs_dict[microstate_label]
        macrostate_samples = random.sample(all_frames, n_samples_per_macrostate)
        samples.append(macrostate_samples)
            
    # samples have to have np.ndarrays in them, not just lists
    samples_ = []
    for x in samples:
        samples_.append(np.asarray(x))
    
    return samples_

In [9]:
msm_300 = MarkovStateModel(lag_time=300)
msm_300.fit(dtrajs)

MSM contains 32 strongly connected components above weight=0.00. Component 31 selected, with population 99.219682%


MarkovStateModel(ergodic_cutoff='on', lag_time=300, n_timescales=None,
         prior_counts=0, reversible_type='mle', sliding_window=True,
         verbose=True)

In [10]:
msm = msm_300
project = 'msm_300'

statdist = msm.populations_
relative_counts = msm.countsmat_.sum(0) / np.sum(msm.countsmat_)

plt.figure()
plt.scatter(statdist, relative_counts)
plt.xlabel('MSM stationary distribution')
plt.ylabel('Relative counts')
plt.savefig('sanity_check_%s.png' %project, dpi=300)
plt.close()

In [11]:
msm_300_p2 = MarkovStateModel(lag_time=300, ergodic_cutoff=0.2)
msm_300_p2.fit(dtrajs)

MSM contains 595 strongly connected components above weight=0.20. Component 487 selected, with population 83.693805%


MarkovStateModel(ergodic_cutoff=0.2, lag_time=300, n_timescales=None,
         prior_counts=0, reversible_type='mle', sliding_window=True,
         verbose=True)

In [3]:
msm_300_p1 = MarkovStateModel(lag_time=300, ergodic_cutoff=0.1)
msm_300_p1.fit(dtrajs)

MSM contains 337 strongly connected components above weight=0.10. Component 314 selected, with population 91.158361%


MarkovStateModel(ergodic_cutoff=0.1, lag_time=300, n_timescales=None,
         prior_counts=0, reversible_type='mle', sliding_window=True,
         verbose=True)

In [13]:
msm = msm_300_p1
project = 'msm_300_p1'

statdist = msm.populations_
relative_counts = msm.countsmat_.sum(0) / np.sum(msm.countsmat_)

plt.figure()
plt.scatter(statdist, relative_counts)
plt.xlabel('MSM stationary distribution')
plt.ylabel('Relative counts')
plt.savefig('sanity_check_%s.png' %project, dpi=300)
plt.close()

In [17]:
msm = msm_300_p2
project = 'msm_300_p2'

statdist = msm.populations_
relative_counts = msm.countsmat_.sum(0) / np.sum(msm.countsmat_)

plt.figure()
plt.scatter(statdist, relative_counts)
plt.xlabel('MSM stationary distribution')
plt.ylabel('Relative counts')
plt.savefig('sanity_check_%s.png' %project, dpi=300)
plt.close()

In [18]:
# p1 is good

In [19]:
# 20 and 50 pcca - sample - cluster

In [19]:
pcca_300_p1_20 = PCCAPlus.from_msm(msm_300_p1, n_macrostates=20)
dtrajs_300_p1_20 = pcca_300_p1_20.transform(dtrajs)

Optimization terminated successfully.
         Current function value: -19.134808
         Iterations: 42
         Function evaluations: 3665


In [5]:
# save metastable_micro_composition for plotting of microstates on tica surface

In [20]:
msmbuilder_pcca_object = pcca_300_p1_20
msmbuilder_msm = msm_300_p1

metastable_sets = []

# make mestable_sets
for x in range(msmbuilder_pcca_object.n_macrostates):
    metastable_sets.append([])
for x in range(len(msmbuilder_pcca_object.microstate_mapping_)):
    metastable_sets[msmbuilder_pcca_object.microstate_mapping_[x]].append(x)

metastable_sets2 = []
    
for metastable_set in metastable_sets:
    all_frames = []
    for microstate in metastable_set:
        microstate_label = msmbuilder_msm.state_labels_[microstate]
        all_frames.append(microstate_label)
    metastable_sets2.append(all_frames)
    
metastable_micro_composition = metastable_sets2    

In [36]:
np.save('metastable_micro_composition', metastable_micro_composition)

In [7]:
# transition path theory for the RIP presentation

In [6]:
metastable_micro_composition = np.load('metastable_micro_composition.npy')

In [9]:
from msmbuilder import tpt

In [21]:
# first do - into macrostates 19 and 8 - 1ZKK-like, from any others
sources = []
sinks = []

for i in range(20):
    if i == 8 or i == 19:
        sinks += metastable_sets[i]
    else:
        sources += metastable_sets[i]

In [16]:
len(sources)

1605

In [17]:
len(sinks)

552

In [22]:
net_flux = msmbuilder.tpt.net_fluxes(sources, sinks, msm_300_p1)

In [23]:
net_flux

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [24]:
top_path = msmbuilder.tpt.top_path(sources, sinks, net_flux)

In [25]:
top_path

(array([ 415, 1913]), 9.4423110905018568e-05)

In [26]:
x = 415

for i in range(20):
    if x in metastable_sets[i]:
        print(i)

16


In [27]:
x = 1913

for i in range(20):
    if x in metastable_sets[i]:
        print(i)

19


In [ ]:
# this wasn't very useful

In [28]:
# let's do from 17 to 19, then from 19 to 17

In [30]:
sources = metastable_sets[17]
sinks = metastable_sets[19]

In [29]:
net_flux = msmbuilder.tpt.net_fluxes(metastable_sets[17], metastable_sets[19], msm_300_p1)

In [31]:
top_path = msmbuilder.tpt.top_path(sources, sinks, net_flux)

In [32]:
top_path

(array([ 181, 1812]), 3.8121033501227898e-05)

In [33]:
x = 181

for i in range(20):
    if x in metastable_sets[i]:
        print(i)

17


In [34]:
x = 1812

for i in range(20):
    if x in metastable_sets[i]:
        print(i)

19


In [36]:
paths = msmbuilder.tpt.paths(sources, sinks, net_flux, num_paths=20)

In [37]:
micro_macro_dict = dict()

for i in range(2500):
    for j in range(20):
        if i in metastable_sets[j]:
            micro_macro_dict[i] = j
    if i not in micro_macro_dict:
        micro_macro_dict[i] = 20 

In [48]:
paths2 = []

for path in paths[0]:
    paths2.append([])
    for microstate in path:
        paths2[-1].append(micro_macro_dict[microstate])

In [49]:
paths2

[[17, 19],
 [17, 19],
 [17, 19],
 [17, 19],
 [17, 19],
 [17, 19],
 [17, 19],
 [17, 15, 19],
 [17, 19],
 [17, 19],
 [17, 19],
 [17, 19],
 [17, 15, 19],
 [17, 19],
 [17, 19],
 [17, 19],
 [17, 19],
 [17, 19],
 [17, 19],
 [17, 19]]

In [50]:
paths

([array([ 181, 1812]),
  array([619,  35]),
  array([1615, 1916]),
  array([725, 645]),
  array([ 181, 1573]),
  array([ 725, 1270]),
  array([725, 571]),
  array([ 105, 1631, 1855]),
  array([1152, 1301]),
  array([181, 118]),
  array([ 725, 1573]),
  array([1066, 1124]),
  array([1152, 2132, 1301]),
  array([1001, 1897]),
  array([725, 958]),
  array([  24, 1874]),
  array([  73, 1547]),
  array([1615, 1544]),
  array([1113, 1547]),
  array([1767, 1573])],
 array([  3.81210335e-05,   3.05087245e-05,   2.44059955e-05,
          2.21656941e-05,   1.54259386e-05,   1.40984432e-05,
          1.17736136e-05,   1.16520711e-05,   1.10909652e-05,
          1.10614024e-05,   1.09775466e-05,   9.06269765e-06,
          8.18987381e-06,   7.62555648e-06,   7.14493359e-06,
          6.95407829e-06,   6.69314150e-06,   6.49877037e-06,
          5.85105639e-06,   5.60879307e-06]))

In [ ]:
# save meta_sets for rmsd analysis

In [9]:
def metastable_sets(msmbuilder_pcca_object, msmbuilder_msm, dtrajs=None):
    import random
    import numpy as np
    
    # make dtrajs_dict
    dtrajs_dict = dict()
    traj_index = 0

    for traj in dtrajs:
        frame_index = 0
        for frame in traj:
            if frame in dtrajs_dict:
                dtrajs_dict[frame].append((traj_index, frame_index))
            else:
                dtrajs_dict[frame] = [(traj_index, frame_index)]
            frame_index += 1    
        traj_index += 1    

    metastable_sets = []
    samples = []
    
    # make mestable_sets
    for x in range(msmbuilder_pcca_object.n_macrostates):
        metastable_sets.append([])
    for x in range(len(msmbuilder_pcca_object.microstate_mapping_)):
        metastable_sets[msmbuilder_pcca_object.microstate_mapping_[x]].append(x)

    metastable_sets2 = []
    
    for metastable_set in metastable_sets:
        all_frames = []
        for microstate in metastable_set:
            microstate_label = msmbuilder_msm.state_labels_[microstate]
            all_frames += dtrajs_dict[microstate_label]
        metastable_sets2.append(all_frames)
        
    return metastable_sets2

In [10]:
meta_sets = metastable_sets(pcca_300_p1_20, msm_300_p1, dtrajs=dtrajs)

In [11]:
np.save('meta_sets_model5', meta_sets)

In [25]:
pcca_300_p1_50 = PCCAPlus.from_msm(msm_300_p1, n_macrostates=50)
dtrajs_300_p1_50 = pcca_300_p1_50.transform(dtrajs)

In [32]:
samples_300_p1_20_100per = macrostate_sample_equal(msm_300_p1, pcca_300_p1_20, n_samples_per_macrostate=100, dtrajs=dtrajs)

In [33]:
samples_300_p1_50_100per = macrostate_sample_equal(msm_300_p1, pcca_300_p1_50, n_samples_per_macrostate=100, dtrajs=dtrajs)

In [34]:
np.save('samples_300_p1_20_100per', samples_300_p1_20_100per)

In [35]:
np.save('samples_300_p1_50_100per', samples_300_p1_50_100per)

In [6]:
# back here to make MSM figure of msm_300_p1_20 and get stat dist

In [51]:
msm_300_p1_20 = pyemma.msm.estimate_markov_model(dtrajs_300_p1_20, lag=300)

In [10]:
plt.figure()
pyemma.plots.plot_markov_model(msm_300_p1_20, arrow_labels=None)
plt.savefig('unitime_2500_msm_300_p1_20.png', dpi=300)
plt.close()

In [6]:
plt.figure()
plt.spy(msm_300_p1_20.P)
plt.savefig('msm_300_p1_20_P.png', dpi=300)
plt.close()

In [9]:
msm_300_p1_20.stationary_distribution

array([ 0.00118174,  0.00377706,  0.00210663,  0.00714451,  0.01740405,
        0.00055486,  0.00384246,  0.00736303,  0.00332306,  0.00149088,
        0.00713653,  0.00396535,  0.00236742,  0.00253077,  0.08525878,
        0.14827181,  0.09523633,  0.19014794,  0.12703765,  0.28985913])

In [11]:
pwd

'/Users/rafalpwiewiora/repos/MSM/11707_11709_SET8_apo/msm_pipeline/dist/unitime_cluster/2500'

In [7]:
# get sparsity of the P

In [8]:
msm_300_p1_20_spy = np.zeros((20,20))

for x in range(20):
    for y in range(20):
        if msm_300_p1_20.P[x,y] > 0:
            msm_300_p1_20_spy[x,y] = 1

In [9]:
msm_300_p1_20_spy

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  1.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,
         0.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  

In [52]:
msm_300_p1

MarkovStateModel(ergodic_cutoff=0.1, lag_time=300, n_timescales=None,
         prior_counts=0, reversible_type='mle', sliding_window=True,
         verbose=True)

In [62]:
msm_300_p1.timescales_[:10]

array([ 40503.31498628,  28101.95160109,  16998.80222111,  13028.22701654,
        12805.61116037,  11350.24276006,  11054.49583093,  10978.46026308,
        10813.34670004,  10379.52451386])

In [54]:
# 100 microsecond simulation from the msm for RIP presentation

# we're gonna plot macrostate index vs time, and the trajectory on the tIC0-tIC1 surface

In [53]:
sequence_of_states = msm_300_p1.sample_discrete(n_steps=2000)

In [55]:
sequence_of_states2 = []

for x in sequence_of_states:
    sequence_of_states2.append([x])

In [56]:
# MSMBUILDER ONLY - SAMPLES EQUAL AMOUNTS OF FRAMES / MACROSTATE; 
def macrostate_sample_HACKED(msmbuilder_msm, n_samples_per_macrostate=1, dtrajs=None):
    import random
    import numpy as np
    
    # make dtrajs_dict
    dtrajs_dict = dict()
    traj_index = 0

    for traj in dtrajs:
        frame_index = 0
        for frame in traj:
            if frame in dtrajs_dict:
                dtrajs_dict[frame].append((traj_index, frame_index))
            else:
                dtrajs_dict[frame] = [(traj_index, frame_index)]
            frame_index += 1    
        traj_index += 1    

    metastable_sets = sequence_of_states2
    samples = []
    
    # make mestable_sets
    #for x in range(msmbuilder_pcca_object.n_macrostates):
     #   metastable_sets.append([])
    #for x in range(len(msmbuilder_pcca_object.microstate_mapping_)):
    #    metastable_sets[msmbuilder_pcca_object.microstate_mapping_[x]].append(x)

    # sample
    for metastable_set in metastable_sets:
        all_frames = []
        for microstate in metastable_set:
            microstate_label = microstate
            all_frames += dtrajs_dict[microstate_label]
        macrostate_samples = random.sample(all_frames, n_samples_per_macrostate)
        samples.append(macrostate_samples)
            
    # samples have to have np.ndarrays in them, not just lists
    samples_ = []
    for x in samples:
        samples_.append(np.asarray(x))
    
    return samples_

In [57]:
samples = macrostate_sample_HACKED(msm_300_p1, dtrajs=dtrajs)

In [58]:
len(samples)

2000

In [59]:
np.save('samples_simulation', samples)

In [60]:
# convert trajectory to macrostate indexes and plot

In [63]:
msmbuilder_pcca_object = pcca_300_p1_20
msmbuilder_msm = msm_300_p1

metastable_sets = []

# make mestable_sets
for x in range(msmbuilder_pcca_object.n_macrostates):
    metastable_sets.append([])
for x in range(len(msmbuilder_pcca_object.microstate_mapping_)):
    metastable_sets[msmbuilder_pcca_object.microstate_mapping_[x]].append(x)

metastable_sets2 = []
    
for metastable_set in metastable_sets:
    all_frames = []
    for microstate in metastable_set:
        microstate_label = msmbuilder_msm.state_labels_[microstate]
        all_frames.append(microstate_label)
    metastable_sets2.append(all_frames)
    
metastable_micro_composition = metastable_sets2    

In [64]:
micro_macro_dict = dict()

for i in range(2500):
    for j in range(20):
        if i in metastable_micro_composition[j]:
            micro_macro_dict[i] = j
    if i not in micro_macro_dict:
        micro_macro_dict[i] = 20 

In [65]:
sequence_of_states

array([ 909,  909,  235, ..., 1224, 1794, 1794], dtype=int32)

In [66]:
sequence_of_states_macro = []

for microstate in sequence_of_states:
    sequence_of_states_macro.append(micro_macro_dict[microstate])

In [68]:
20 in sequence_of_states_macro

False

In [69]:
# ok good so doesn't seem like any problems - we don't have any microstates which shouldn't be there

In [71]:
# let's plot this

In [74]:
plt.figure()
plt.plot(sequence_of_states_macro)
plt.xlabel('Time (150 ns / step)')
plt.ylabel('Macrostate index')
plt.savefig('simulation_macro.png', dpi=300)

In [73]:
pwd

'/Users/rafalpwiewiora/repos/MSM/11707_11709_SET8_apo/msm_pipeline/dist/unitime_cluster/2500'